<div style="text-align: center"> <H2> Домашнее задание №14, к лекции "Основы парсинга и работы с API" </H2></div>
<div style='text-align: center'> <H4> по курсу PYDA-15 - Python для анализа данных </H4></div>
</div>
<p>
<div style="text-align: right"> Выполнил: студент группы PAL-4 Константин Лемешок </div>

---

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime, timedelta

### Задание 1

#### Обязательная часть

<div class="alert alert-block alert-success">

Будем парсить страницу со свежеми новостям на <a href='https://habr.com/ru/all/'>habr.com/ru/all/</a>.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
    
    KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида:
    
    <дата> - <заголовок> - <ссылка>

</div>

---

In [2]:
res = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(res.text, 'html.parser')
posts = soup.find_all('article', class_='post')

In [3]:
KEYWORDS = ['Python', 'парсинг', 'data']

In [4]:
monthes = {
    'января': 'January',
    'февраля': 'February',
    'марта': 'March',
    'апреля': 'April',
    'мая': 'May',
    'июня': 'June',
    'июля': 'July',
    'августа': 'August',
    'сентября': 'September',
    'октября': 'October',
    'ноября': 'November',
    'декабря': 'December'
}

In [5]:
# создадим датафрейм для записи результатов
news_table = pd.DataFrame()

for post in posts:
#     отберем посты, которые содержат хотя бы 1 из KEYWORDS
    if any([keyword.lower() in post.text.lower() for keyword in KEYWORDS]):
#         вытащим заголовок и ссылку из каждого выбраного поста
        title = post.find('a', class_='post__title_link').text
        link = post.find('a', class_='post__title_link').get('href')
#         вытащим время публикации поста в текстовом виде
        texttime = post.find('span', class_='post__time').text
#         определим дату
        if 'сегодня' in texttime:
            date = datetime.today().date()
        elif 'вчера' in texttime:
            date = (datetime.now() - timedelta(days=1)).date()            
        else:
# Я не нашел, как конвертировать в формат datetime дату с месяцем на русском языке в РОДИТЕЛЬНОМ падеже
# типа '17 декабря 2020 в 23:50'. import locale, locale.setlocale(locale.LC_ALL, 'ru') работает только
# для месяца в именительном падеже типа '17 декабрь 2020 в 23:50'. Поэтому придется пойти "кривым" путем
# и заменить месяц на английский вариант с использованием соответствующего словаря monthes (см. ячейку выше):
            month_ru = texttime[3:-13]
            for key, value in monthes.items():
                if month_ru == key:
                    month_eng = value
                    break
            texttime = texttime.replace(month_ru, month_eng)[:-8]
            date = datetime.strptime(texttime, '%d %B %Y').date()
#         дополним датафрейм новой строкой
        row = {'date': date, 'title': title, 'link': link}
        news_table = pd.concat([news_table, pd.DataFrame([row])])
news_table

,date,title,link
0,2020-12-20,Шифр Цезаря или как просто зашифровать текст,https://habr.com/ru/post/534058/
0,2020-12-20,«CPPA от GDPR не далеко падает»: в Канаде пыта...,https://habr.com/ru/company/vasexperts/blog/53...
0,2020-12-20,FOSS News №47 – дайджест новостей и других мат...,https://habr.com/ru/post/534012/


---

#### Дополнительная часть (необязательная)

<div class="alert alert-block alert-success">

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами:
    
    <дата> - <заголовок> - <ссылка> - <текст_статьи>

</div>

---

In [6]:
# Для начала вынесем в отдельную функцию определение даты

def pubdate(texttime):
    '''Функция принимает строку со временем публикации новости и преобразует ее в дату в формате DateTime'''
    
    monthes = {
    'января': 'January',
    'февраля': 'February',
    'марта': 'March',
    'апреля': 'April',
    'мая': 'May',
    'июня': 'June',
    'июля': 'July',
    'августа': 'August',
    'сентября': 'September',
    'октября': 'October',
    'ноября': 'November',
    'декабря': 'December'
    }
        
    if 'сегодня' in texttime:
        return datetime.today().date()
    elif 'вчера' in texttime:
        return (datetime.now() - timedelta(days=1)).date()            
    else:
        month_ru = texttime[3:-13]
        for key, value in monthes.items():
            if month_ru == key:
                month_eng = value
                break
        texttime = texttime.replace(month_ru, month_eng)[:-8]
        return datetime.strptime(texttime, '%d %B %Y').date()

In [7]:
# Теперь сразу создадим функцию, которая будет делать все, что надо

def get_news_from_habr_by_keywords(URL, keywords):
    ''' Функция принимает на вход любую страницу (из страниц  habr.com/ru/all/) и набор ключевых слов в виде словаря,
    а на выходе отдает датафрейм из новостей, которые содержат заданные ключевые слова,
    в формате <date>-<title>-<link>-<news text>
    '''
    res = requests.get(URL)
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    
#     создадим датафрейм для записи результатов
    news_table = pd.DataFrame()
    
    for post in posts:
#         получим полный текст новости
        link = post.find('a', class_='post__title_link').get('href')
        fulltext = BeautifulSoup(requests.get(link).text, 'html.parser')
        time.sleep(0.3)
#         проверим, есть ли в полном тексте новости хотя бы 1 из keywords
        if any([keyword.lower() in fulltext.text.lower() for keyword in keywords]):
#             и если да, то вытащим дату, заголовок и текст новости
            texttime = fulltext.find('span', class_='post__time').text
            date = pubdate(texttime)
            title = fulltext.find('span', class_='post__title-text').text
            news_text = fulltext.find('div', class_='post__text').text
#             ну и, конечно, добавим их в датафрейм
            row = {'date': date, 'title': title, 'link': link, 'news text': news_text}
            news_table = pd.concat([news_table, pd.DataFrame([row])])
    return news_table

In [8]:
# Выбираем любую страницу, например № 8, выбираем ключевые слова... и получаем нужный результат

URL = 'https://habr.com/ru/all/page8/'
keywords = ['Python', 'парсинг']

get_news_from_habr_by_keywords(URL, keywords)

,date,title,link,news text
0,2020-12-17,«Матрица рисков» компании. Алгебраическое иссл...,https://habr.com/ru/post/533628/,\n\r\nМне доводилось принимать участие в упраж...
0,2020-12-17,Материалы Avito Design Talk: видео и презентации,https://habr.com/ru/company/avito/blog/533620/,Привет! Недавно прошёл наш митап для дизайнеро...
0,2020-12-17,"Опубликован релиз Sequoia 1.0, реализации Open...",https://habr.com/ru/company/selectel/blog/533622/,\r\nСпустя три года после начала разработки оп...
0,2020-12-17,Дополнительное поле в карточке товара OpenCart,https://habr.com/ru/post/533608/,Статья нашего разработчика из его личного блог...
0,2020-12-17,Проблема статус-кодов HTTP,https://habr.com/ru/post/533606/,Ситуацию с использованием кодов ответов HTTP м...
0,2020-12-17,Как я анализировал свои поездки на такси,https://habr.com/ru/post/533586/,"Всякий раз, как я езжу на такси мне на почту п..."


---

### Задание 2

#### Обязательная часть

<div class="alert alert-block alert-success">

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса <a href='https://www.avast.com/hackcheck/'>Avast Hack Ckeck</a>. Список email-ов задаем переменной в начале кода:
    
    EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами:
    
    <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

</div>

---

### Уже все не так просто, сервис <a href="https://prnt.sc/w6k74g" >начал временами требовать подтвердить</a>, что ты человек. И если ты скрипт и не нажимаешь на чекбокс,  то вместо словаря он отдает null. В итоге можем получить ошибку 'NoneType' object is not subscriptable. Но в этот раз нам повезло...

In [9]:
# Для начала напишем функцию, которая будет получать результаты запроса от сервиса Avast по одному указанному email
# и преобразовывать их в датафрейм:

def get_email_leaks_info(email):
    '''Функция получает на вход один email в виде строки, проверяет его на утечки паролей в разных сервисах
    согласно данным Avast и возвращает DataFrame с перечнем утечек в формате <email>-<date>-<source>-<description>.
    '''
    
    URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    data = {'emailAddresses': [email]}
    headers = {
    'Host': 'identityprotection.avast.com',
    'Connection': 'keep-alive',
    'Content-Length': '32',
    'Accept': 'application/json, text/plain, */*',
    'Vaar-Version': '0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Content-Type': 'application/json;charset=UTF-8',
    'Origin': 'https://www.avast.com',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.avast.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en;q=0.9,ru-UA;q=0.8,ru;q=0.7,uk-UA;q=0.6,uk;q=0.5,en-US;q=0.4'
    }
    
#     достанем данные с сайта и распарсим их в BeautifulSoup
    req = requests.post(URL, json=data, headers=headers)
    
#     преобразуем полученный результат в словарь для последующей итерации по нему
    dictionary = req.json()['breaches']
    
#     создадим пустой датафрейм, куда потом будем добавлять данные
    one_email_leak_table = pd.DataFrame()
    
#     достанем из словаря значения соответствующих ключей и добавим их в строку, а затем и в датафрейм
    for leak in dictionary.values():
        date = leak['publishDate'][:10]
        source = leak['site']
        description = leak['description']
        row = {'email': email, 'date': date, 'source': source, 'description': description}
        one_email_leak_table = pd.concat([one_email_leak_table, pd.DataFrame([row])])
    
#     сделаем паузу, дадим серверу отдохнуть от наших запросов, чтобы он на нас не обиделся
    time.sleep(0.3)

    return one_email_leak_table

In [10]:
# Напишем функцию, которая будет работать сразу со списком адресов, добавляя их в один датафрейм

def get_all_emails_leaks_info(all_emails_list):
    
#     создадим пустой датафрейм, куда потом будем добавлять данные
    all_emails_leak_table = pd.DataFrame()
    
#     выполним поиск утечек для каждого из адресов, и добавим все данные в общий датафрейм all_emails_leak_table
    for email in all_emails_list:
        one_email_leak_table = get_email_leaks_info(email)
        all_emails_leak_table = pd.concat([all_emails_leak_table, one_email_leak_table])
    
    return all_emails_leak_table

In [11]:
# Применяем все написанное на практике, на примере двух моих адресов и адреса преподавателя.
# Обращаем внимание, что если по адресу утечек не было, все равно все работает.

emails = ['regenmacher@gmail.com', 'obulygin91@ya.ru', 'klemeshok@gmail.com']

get_all_emails_leaks_info(emails)

,email,date,source,description
0,regenmacher@gmail.com,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
0,regenmacher@gmail.com,2016-11-01,qip.ru,"In 2011, Russian instant messaging service pro..."
0,regenmacher@gmail.com,2016-10-29,vk.com,Popular Russian social networking platform VKo...
0,regenmacher@gmail.com,2019-02-21,emaildatalist.net,"This is a list of email addresses only, and as..."
0,regenmacher@gmail.com,2018-03-20,myfitnesspal.com,"In February 2018, MyFitnessPal's database was ..."
0,regenmacher@gmail.com,2019-02-28,500px.com,"In July 2018, 500px's database was allegedly b..."
0,regenmacher@gmail.com,2019-05-23,livejournal.com,"In 2017, social network LiveJournal's database..."
0,obulygin91@ya.ru,2019-03-20,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
0,obulygin91@ya.ru,2019-03-07,armorgames.com,"In December 2018, Armor Games' database was al..."
0,obulygin91@ya.ru,2020-07-23,wattpad.com,"In June 2020, the online writing community Wat..."


---

#### Дополнительная часть (необязательная)

<div class="alert alert-block alert-success">

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods, вам поможет метод wall.get.

    GROUP = 'netology'
    TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами:
    
    <дата поста> - <текст поста>

</div>

---

In [12]:
def get_50_last_posts_from_vk_user_or_group(domain):
    '''Функция получает на входе domain (короткий адрес) пользователя или сообщества и отдает на выходе датафрейм
    с 50 последними постами на странице этого пользователя или сообщества, в формате <date>-<text>
    '''
    
#     читаем токен
    with open ('D:\\OneDrive\\!Netology\\!Netology_Pal-4\\01_Python_for_data_analysis\\token.txt', 'r') as file_object:
        TOKEN = file_object.read().strip()
    
#     задаем необходимые переменные и формируем словарь с параметрами запроса согласно документации
#     используемого метода из API https://vk.com/dev/wall.get 
    LASTPOSTS_REQUEST = 'https://api.vk.com/method/wall.get?'
    VERSION = '5.84'
    params = {
        'access_token': TOKEN,
        'v': VERSION,
        'domain': domain,
        'count': 50,
        'filter': 'all'
    }
    
#     обращаемся к API и забираем необходимые данные
    res = requests.get(LASTPOSTS_REQUEST, params)
    
#     переводим полученные данные сначала в словарь, а затем в датафрейм, отобрав при этом только нужные столбцы
    df = pd.DataFrame(res.json()['response']['items'])[['date', 'text']]
    
#     добавим в датафрейм колонку со временем публикации в "человеческом" виде
    df['time'] = pd.to_datetime(df['date'], unit='s')
#     и уберем из датафрейма ненужную теперь колонку 'date'
    df = df[['time', 'text']]
    
    return df

In [13]:
domain = 'netology'

get_50_last_posts_from_vk_user_or_group(domain)

,time,text
0,2020-12-14 10:05:01,Наш предновогодний digital-марафон продолжаетс...
1,2020-12-20 07:53:00,"Предсказать будущее невозможно, но можно его п..."
2,2020-12-19 13:24:00,"Data Science митап ВТБ, конкурс IT-проектов и ..."
3,2020-12-19 08:10:00,"Написать письмо коллеге, пройти собеседование ..."
4,2020-12-18 14:22:00,"В эфире #пятничнаябеседка. Здесь мы обсуждаем,..."
5,2020-12-18 07:33:00,🔥 Запустили бесплатный курс «Как стать SEO‑спе...
6,2020-12-17 14:46:00,Google Docs — это целый набор средств редактир...
7,2020-12-17 07:54:00,Приглашение на работу в Кремниевую долину — зв...
8,2020-12-16 13:15:03,У нас сегодня Нето-викторина 🤩\n\nМы подготови...
9,2020-12-16 07:23:00,🏠 Запустили бесплатный онлайн-курс «Дизайн инт...
